In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, GenerationConfig

transformers.set_seed(42)
torch.cuda.empty_cache()

# 1. Load Models from Config

In [2]:
config = AutoConfig.from_pretrained(
  'mosaicml/mpt-7b',
  trust_remote_code=True,
  revision='main'
)
# config.attn_config['attn_impl'] = 'triton'

tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
model = AutoModelForCausalLM.from_pretrained(
  config._name_or_path,
  config=config,
  trust_remote_code=True,
  revision='main'
).to(device='cuda:0')

/home/han/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b/053e1a33a6e7043aefaa3f5d13c48269a5511cff/attention.py:157: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 24.00 GiB total capacity; 23.02 GiB already allocated; 0 bytes free; 23.02 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Based on the model architecture, we find that
- MPT uses MPTForCausalLM module.
- LoRA fine-tuning target module is `Wqkv` module of the MultiheadAttention block.

# 2. Text Generation

In [ ]:
generation_config_map = {
    'max_new_tokens': 100,
    'do_sample': True,
    'top_k': 0,
    'temperature': 0.7,
    'return_dict_in_generate': True,
    'output_scores': True,
    'early_stopping': True,
    'pad_token_id': tokenizer.eos_token_id
}
generation_config = GenerationConfig.from_pretrained(
    config._name_or_path,
    **generation_config_map    
)

In [ ]:
input_prompt = "It is a good day to die."
input_ids = tokenizer.encode(
    input_prompt,
    return_tensors='pt'
).to('cuda:0')

outputs = model.generate(
    input_ids,
    generation_config
)
decoded_text = tokenizer.decode(outputs[0][0], skip_special_tokens=True)

In [ ]:
print(decoded_text)

It is a good day to die.
I don’t know what I did to deserve this treatment.
It’s not like I was perfect.
I’m a junkie, a thief, a liar, and I have a mental disorder.
A family member of mine just died.
If I am to die, all I want to do is die alone.
I don’t want to see the faces of my family members.
I don’t want to hear their voices.
I don’
